In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation, add, Flatten, AveragePooling1D, concatenate
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

width=32
height = 32

from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.utils import to_categorical

In [33]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [38]:
num_classes = 10
y_train = tf.keras.uptils.to_categorical(y_train)
y_test = tf.keras.uptils.to_categorical(y_test)

AttributeError: module 'tensorflow.keras' has no attribute 'uptils'

In [39]:
num_classes = 10
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

In [40]:
validation_img, validation_labels = x_train[:500], y_train_oh[:500]
train_imgs, validation_labels = x_train[500:], y_train_oh[500:]

In [41]:
train_imgs.shape

(49500, 32, 32, 3)

In [42]:
def inception(x, filters):       # 필터 값 4개를 입력받음
    pre_layer = x
    f1, f2, f3, f4 = filters
    
    conv1 = Conv2D(f1, kernel_size=(1,1), padding='same', activation='relu')(pre_layer)
    
    conv2 = Conv2D(f4, kernel_size=(1,1), padding='same', activation='relu')(pre_layer)
    conv2 = Conv2D(f2, kernel_size=(3,3), padding='same', activation='relu')(conv2)
    
    conv3 = Conv2D(f4, kernel_size=(1,1), padding='same', activation='relu')(pre_layer)
    conv3 = Conv2D(f3, kernel_size=(5,5), padding='same', activation='relu')(conv3)
    
    max_pool = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(pre_layer)
    max_pool = Conv2D(f4, kernel_size=(1,1), padding='same')(max_pool)
    
    concat = concatenate([conv1, conv2, conv3, max_pool], axis=1)
    return concat

In [43]:
input_shape = x_train[0].shape
inputs = Input(shape=input_shape)

x = Conv2D(64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(inputs)
x = BatchNormalization()(x)
x = Conv2D(192, kernel_size=(3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)

x = inception(x, [64, 128, 32, 32])
x = inception(x, [128, 192, 96, 64])
MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)

x = inception(x, [192, 208, 48, 64])

aux1 = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
aux1 = Conv2D(128, kernel_size=(1,1), padding='same', activation='relu')(x)
aux1 = Flatten(aux1)
aux1 = Dense(512, activation='relu')(aux1)
aux1 = Dense(10, activation='softmax')(aux1)


x = inception(x, [160, 224, 64, 4])

x = inception(x, [128, 256, 64, 64])
x = inception(x, [112, 288, 64, 64])


aux2 = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
aux2 = Conv2D(128, kernel_size=(1,1), padding='same', activation='relu')(aux2)
aux2 = Flatten(aux2)
aux2 = Dense(512, activation='relu')(aux2)
aux2 = Dense(10, activation='softmax')(aux2)



x = inception(x, [256, 320, 128, 128])
x = inception(x, [256, 320, 128, 128])
x = inception(x, [112, 288, 64, 64])


x = AveragePooling2D(pool_size(4,4), padding='valid')(x)
x = Dropout(0.4)(x)
x = Flatten()(x)

output = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs=[aux1, aux2, outputs])

model.summary()



ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 16, 16, 64), (None, 16, 16, 128), (None, 16, 16, 32), (None, 16, 16, 32)]